In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, fixed

# Create The Model For Our Algorithm
To be able to calculate teams I would like to be able to input "Team1", "Team2" into the algorithm and have the algorithm spit out the probability that each team will win the game.

To do this we must set up The game data so that the supervised machine learning algorithm we use can be trained using game data rather than seasonal data.

In [2]:
current_season_data = pd.read_pickle("new_season_detailed_results")
past_season_game_data = pd.read_pickle("past_season_detailed_results")
past_tourney_game_data = pd.read_pickle("past_tourney_detailed_results")
team_seasonal_record_data = pd.read_pickle("calculated_past_team_seasonal_data")

In [3]:
current_season_data.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
65872,2016,11,1104,77,1244,64,H,0,29,57,...,19,19,26,12,27,6,16,7,4,25
65873,2016,11,1105,68,1408,67,A,1,25,64,...,27,16,26,18,30,11,19,6,7,21
65874,2016,11,1112,79,1334,61,H,0,24,61,...,19,13,19,5,23,9,9,3,1,25
65875,2016,11,1115,58,1370,56,A,0,20,55,...,16,16,28,10,31,12,15,5,0,17
65876,2016,11,1116,86,1380,68,H,0,32,66,...,12,20,28,7,21,9,17,8,5,22


In [4]:
past_season_game_data.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [5]:
past_tourney_game_data.head()

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot,Wfgm,Wfga,...,Lfga3,Lftm,Lfta,Lor,Ldr,Last,Lto,Lstl,Lblk,Lpf
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [6]:
team_seasonal_record_data.head()

,Team_Name,Season,Win_Percentage,Points_Per_Game,Percent_FG_Made,Percent_FT_Made,Percent_3s_Made,OR_Per_Game,DR_Per_Game,Assists_Per_Game,TO_Per_Game,Steals_Per_Game,Blocks_Per_Game,Fouls_Per_Game
Team_Id,,,,,,,,,,,,,,
1102,Air Force,2003,0.428571,57.250000,0.481149,0.651357,0.375643,6.178571,16.000000,13.000000,11.428571,5.678571,1.785714,18.750000
1103,Akron,2003,0.481481,78.777778,0.486074,0.736390,0.338710,10.962963,20.000000,15.222222,12.629630,6.629630,2.333333,19.851852
1104,Alabama,2003,0.607143,69.285714,0.420362,0.709898,0.320144,15.035714,29.928571,12.107143,13.285714,7.535714,3.785714,18.035714
1105,Alabama A&M,2003,0.269231,71.769231,0.395755,0.705986,0.364815,7.461538,13.307692,14.538462,18.653846,4.961538,2.076923,20.230769
1106,Alabama St,2003,0.464286,63.607143,0.423773,0.646421,0.346154,11.250000,22.107143,11.678571,17.035714,8.392857,3.142857,18.178571


# Reformat The Game Data
In this section we will reformat the game data so that it is usable in our regression machine learning model.

The first thing we must do is import our **ncaa_helper** and use the **convert_season_tourney_data()** function to convert our game data to the appropriate form so that we can input it into our machine learning model.

In [7]:
import ncaa_helper as nh

In [8]:
model = nh.convert_season_tourney_data(past_season_game_data, past_tourney_game_data, team_seasonal_record_data)
model.head()

,Season,Daynum,Numot,Wteam,Wscore,Wor,Wdr,Wast,Wto,Wstl,Wblk,Wpf,Wfgp,Wftp,Wfgp3,Win_Percentage,Win
index,,,,,,,,,,,,,,,,,
0,2003,10,0,1104,68,14,24,13,23,7,1,22,0.465517,0.611111,0.214286,0.607143,1
1,2003,18,0,1104,82,12,26,13,13,7,3,14,0.489796,0.705882,0.500000,0.607143,1
2,2003,21,0,1104,80,10,24,16,13,10,6,13,0.457627,0.782609,0.333333,0.607143,1
3,2003,25,0,1104,54,17,26,12,13,6,6,21,0.280702,0.714286,0.280000,0.607143,1
4,2003,29,0,1104,89,16,30,16,10,10,4,17,0.485714,0.800000,0.409091,0.607143,1


# Create The Train/Test Split
In this section we create the train/test split in our data so that we can test the accuracy of different models and also compute the effectiveness of each feature(column) in our data.

In [9]:
from sklearn import cross_validation

In [10]:
feature_columns = model.columns[0:len(model.columns) - 1]
target_column = "Win"

X = model[feature_columns]
y = model[target_column]

# Split data
Xtrain, Xtest, ytrain, ytest = cross_validation.train_test_split(X, y)

# Calculate Importances Of Features
In this section we use the RandomForestClassifier to calculate the importance of features and remove the features that are not significant. This will reduce our chances of overfitting the data to our chosen model.

In [11]:
from sklearn import ensemble

In [12]:
forest_class = ensemble.RandomForestClassifier()
forest_class.fit(Xtrain, ytrain)
forest_ytest = forest_class.predict(Xtest)

Output the accuracy of the forest model of our training data.

In [13]:
print(forest_class.score(Xtrain, ytrain))

0.990622310294


Here we output the importances of each column and analyze which columns are important and which are not.

In [14]:
def get_importances(X, forest_class):
    return pd.DataFrame(list(zip(X.columns, forest_class.feature_importances_))).sort_values(1, ascending=False)

In [15]:
importances = get_importances(X, forest_class)
importances

,0,1
12,Wfgp,0.157298
4,Wscore,0.134278
6,Wdr,0.124031
15,Win_Percentage,0.090375
11,Wpf,0.087228
14,Wfgp3,0.056737
9,Wstl,0.047038
13,Wftp,0.047030
8,Wto,0.045330
3,Wteam,0.040113


# Fit Our First Model
In this section we try to fit the data to a model and analyze the accuracy of the fit. We will also make sure to remove the least important features from our data before fitting to a model.

In [16]:
from sklearn import linear_model

In [17]:
logistic_class = linear_model.LogisticRegression(solver="newton-cg")
logistic_class.fit(Xtrain, ytrain)
logistic_predict = logistic_class.predict(Xtest)

Output the score of the logistic regression model.

In [18]:
logistic_class.score(Xtest, ytest)

0.84945655437458711

# Remove Unnecessary Features
In this section we will remove the unnecessary features that we analyzed above to make sure our model is accurate and we are not overfitting the data.

Calculate all columns with values higher than our threshold value of 0.05

In [19]:
threshold = 0.05
new_columns = list(importances[importances[1] > threshold][0])
# new_columns = list(importances[0:len(importances) - 4][0])

Refit model with new columns and test the accuracy of our logistic model.

In [20]:
X = model[new_columns]
y = model["Win"]

Xtrain, Xtest, ytrain, ytest = cross_validation.train_test_split(X, y)

In [21]:
logistic_class = linear_model.LogisticRegression(solver="newton-cg")
logistic_class.fit(Xtrain, ytrain)
logistic_predict = logistic_class.predict(Xtest)
logistic_class.score(Xtest, ytest)

0.82129346063772291

# Analyze Column Importance Again

In [22]:
forest_class = ensemble.RandomForestClassifier()
forest_class.fit(Xtrain, ytrain)
forest_ytest = forest_class.predict(Xtest)

In [23]:
get_importances(X, forest_class)

,0,1
0,Wfgp,0.241778
1,Wscore,0.199946
3,Win_Percentage,0.169175
2,Wdr,0.147237
4,Wpf,0.126785
5,Wfgp3,0.115079


In [24]:
# pd.DataFrame(list(zip(X.columns, np.transpose(clf.coef_))))

# Create Our Prediction Function
In this section we will explore the prediction function in the **ncaa_helper** module.

In [25]:
teams = pd.read_pickle("teams")
reg_season_2016 = pd.read_pickle("new_season_detailed_results")

In [26]:
team_season_data = nh.calc_year_data(2016, reg_season_2016, teams)
team_season_data[team_season_data.Team_Name == "Duke"]

,Team_Name,Season,Win_Percentage,Points_Per_Game,Percent_FG_Made,Percent_FT_Made,Percent_3s_Made,OR_Per_Game,DR_Per_Game,Assists_Per_Game,TO_Per_Game,Steals_Per_Game,Blocks_Per_Game,Fouls_Per_Game
Team_Id,,,,,,,,,,,,,,
1181,Duke,2016,0.69697,81.454545,0.458925,0.722222,0.386852,16.212121,33.575758,13.484848,9.878788,7.727273,4.181818,16.636364


In [27]:
model.head()

,Season,Daynum,Numot,Wteam,Wscore,Wor,Wdr,Wast,Wto,Wstl,Wblk,Wpf,Wfgp,Wftp,Wfgp3,Win_Percentage,Win
index,,,,,,,,,,,,,,,,,
0,2003,10,0,1104,68,14,24,13,23,7,1,22,0.465517,0.611111,0.214286,0.607143,1
1,2003,18,0,1104,82,12,26,13,13,7,3,14,0.489796,0.705882,0.500000,0.607143,1
2,2003,21,0,1104,80,10,24,16,13,10,6,13,0.457627,0.782609,0.333333,0.607143,1
3,2003,25,0,1104,54,17,26,12,13,6,6,21,0.280702,0.714286,0.280000,0.607143,1
4,2003,29,0,1104,89,16,30,16,10,10,4,17,0.485714,0.800000,0.409091,0.607143,1


In [28]:
def rename_cols(team_data):
    return team_data.rename(columns={
            "Points_Per_Game": "Wscore",
            "OR_Per_Game": "Wor",
            "DR_Per_Game": "Wdr",
            "Assists_Per_Game": "Wast",
            "TO_Per_Game": "Wto",
            "Steals_Per_Game": "Wstl",
            "Blocks_Per_Game": "Wblk",
            "Fouls_Per_Game": "Wpf",
            "Percent_FG_Made": "Wfgp",
            "Percent_FT_Made": "Wftp",
            "Percent_3s_Made": "Wfgp3"
        })

In [29]:
def predict(team):
    team = team_season_data[team_season_data.Team_Name == team]
    team = rename_cols(team)
    team["Numot"] = 0

    return logistic_class.predict_proba(team[new_columns])

In [30]:
#class imbalance
# fg team 1 and fg team 2
# flip data using split test
print(predict("Duke"))
print(predict("Gonzaga"))

[[ 0.01358815  0.98641185]]
[[ 0.00340438  0.99659562]]


In [31]:
interact(predict, team=tuple(teams["Team_Name"][60:200:10]), __manual=True);